In [ ]:
from copy import deepcopy
import numpy as np
from tqdm import tqdm
from desk_optim_util import Group, DesksClass

In [ ]:
# Parameters (touch here)

# Desks are available if they are not -1
# Island: Set of adjacent numbers except -1
# Ocean: -1
desks_matrix = [
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 47],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 48],
    [20, 23, -1, 27, 31, -1, 35, 39, -1, 43, 49],
    [-1, -1, -1, 28, 32, -1, 36, 40, -1, 44, 50],
    [21, 25, -1, 29, 33, -1, 37, 41, -1, 45, 51],
    [22, 26, -1, 30, 34, -1, 38, 42, -1, 46, 52],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 53],
]

group_list = [
    Group("Nishida G", "西", [37, 38, 41, 42], 4),
    Group("Haruno G", "春", [25, 26, 30, 52, 53], 0),
    Group("Hayashi G", "林", [27, 28, 29, 31], 1),
    Group("Naitoh G", "内", [43, 44, 45], 0),
    Group("Suzuki G", "鈴", [47, 48, 49], 0),
    Group("Kitazawa G", "北", [32, 46], 0),
    Group("Hagura G", "羽", [51], 1),
    Group("Kida G", "黄", [33], 0),
    Group("Ban G", "番", [39], 0),
    Group("Nakano G", "中", [34], 0),
]

# Beam search setting
n_beam = 100
n_search = 10
n_iterate = 200

In [ ]:
np.random.seed(0)
desks_cls = DesksClass(desks_matrix, group_list)
desk_set_remained = desks_cls.desk_set.copy()
for g in group_list:
    desk_set_remained -= g.existing_desks

desk_set_remained = list(desk_set_remained)
new_desk_groups = [
    list(g.existing_desks) + [desk_set_remained.pop() for _ in range(g.n_new)]
    for g in group_list
]
new_desk_groups_beam_old = []
for i in range(n_beam):
    new_desk_groups_i, desk_set_remained_i = desks_cls.change(deepcopy(new_desk_groups), deepcopy(desk_set_remained))
    new_desk_groups_beam_old.append([desks_cls.get_loss(deepcopy(new_desk_groups_i)), new_desk_groups_i, desk_set_remained_i])

for _ in tqdm(range(n_iterate)):
    new_desk_groups_beam = []
    for _, new_desk_groups, desk_set_remained in new_desk_groups_beam_old:
        for _ in range(n_search):
                new_desk_groups_i, desk_set_remained_i = desks_cls.change(deepcopy(new_desk_groups), deepcopy(desk_set_remained))
                new_desk_groups_beam.append(
                    [desks_cls.get_loss(deepcopy(new_desk_groups_i)), new_desk_groups_i, desk_set_remained_i]
                )

    new_desk_groups_beam = sorted(new_desk_groups_beam, key=lambda x: x[0])[:n_beam]
    new_desk_groups_beam_old = deepcopy(new_desk_groups_beam)

In [ ]:
for losses, res_desk_groups, _ in new_desk_groups_beam[:2]:
    print(losses)
    desks_matrix_cp = deepcopy(desks_matrix)
    for i in range(desks_cls.h):
        for j in range(desks_cls.w):
            if desks_matrix_cp[i][j] == -1: continue
            for desks_group, group in zip(res_desk_groups, group_list):
                if desks_matrix_cp[i][j] in desks_group:
                    desks_matrix_cp[i][j] = group.icon

    for desks_row in desks_matrix_cp: print(desks_row)
    print()